In [12]:
import pandas as pd
import geopandas as gpd
import numbers
import numpy as np

Read in the integrated dataset

In [70]:
df = pd.read_csv('../NZ_Data/integrated/modified_integrated.csv')

Read in the Statistical Area 1 shapefile and convert the SA1 codes to integers.

In [76]:
shape = gpd.read_file('../QGIS/statsnz-statistical-area-1-2018-generalised-SHP/statistical-area-1-2018-generalised.shp')

In [78]:
lst = list(shape['SA12018_V1'])
for i in range(0,len(lst)):
    lst[i] = int(lst[i])

In [80]:
shape['SA12018_V1'] = lst

Read in the SA2 codes of areas in Christchurch

In [82]:
# SA2 codes
sa2 = list(set([316400,316500,316600,316700,316800,316900,317000,317100,317200,317300,317400,317500,317600,317700,
       317800,317900,318000,318200,318300,318400,318500,318600,318700,318800,318900,319000,319100,319200,
       319300,319400,319500,319600,319700,319800,319900,320000,320100,320200,320300,320400,320500,320600,
       320700,320800,320900,321000,321100,321200,321300,321400,321500,321600,321700,321800,321900,322000,
       322100,322200,322300,322400,322500,322600,322700,322800,322900,323000,323100,323100,323200,323300,
       323400,323500,323600,323700,323800,323900,324000,324200,324300,324400,324500,324600,324700,324800,
       324900,325000,325100,325200,325300,325400,325500,325600,325700,325800,325900,326000,326100,326200,
       326300,326400,326500,326600,326700,326800,326900,327000,327100,327200,327300,327400,327500,327600,
       327700,327800,327900,328000,328100,328200,328300,328400,328500,328600,328700,328800,328900,329000,
       329100,329200,329300,329400,329500,329600,329700,329800,329900,330000,330100,330200,330300,330400,
       330500,330600,330700,330800,330900,331000,331200,331300,331400,331500,331600,331700,331800,331900,
       332000,332100,332400,332700,318100,324100,331100,326200]))

In [84]:
df2 = pd.DataFrame(sa2, columns=['SA2'])

In [86]:
df3 = pd.merge(df, df2, left_on='SA22018_code', right_on='SA2', how='right', validate='m:1')

In [88]:
df3 = df3.drop(['Unnamed: 0', 'SA2'], axis = 1)

In [113]:
df3.to_csv('../NZ_Data/case_study/christchurch/cc_data.csv')

Merge the shapefile with the christchurch data to make the geojson export 

In [115]:
gdf = shape.merge(df3, how='inner', left_on = 'SA12018_V1', right_on = 'SA12018_V1_00')

In [117]:
gdf = gdf.drop(['SA12018_V1_00'], axis = 1)

In [119]:
# remove redundant columns
gdf = gdf.loc[:, ~gdf.columns.duplicated(keep='first')]

In [121]:
gdf.to_file('../NZ_Data/case_study/christchurch/cc_data.geojson', driver='GeoJSON')

Get the total number of industrial buildings in each SA1 area

In [107]:
cols = [x for x in df3.columns if 'Ind22' in x]

In [109]:
total_ind = []
for index, row in df3.iterrows():
    indsum = 0
    for i in cols:
        if not np.isnan(row[i]):
            indsum = indsum + row[i]
    total_ind.append(indsum)

In [111]:
df3['Ind22_totals'] = total_ind